In [1]:
import numpy as np
from scipy.stats import binom
import matplotlib.pyplot as plt

In [5]:
from option import VanillaOption, call_payoff, put_payoff

In [6]:
def american_binomial_prob4(option, spot, rate, vol, div, steps):
    strike = option.strike
    expiry = option.expiry
    call_t = 0.0
    spot_t = 0.0
    h = expiry / steps
    num_nodes = steps + 1
    #u = np.exp((rate - div) * h + vol * np.sqrt(h))
    #d = np.exp((rate - div) * h - vol * np.sqrt(h))
    u = 1.3
    d = 0.8
    pstar = (np.exp(rate * h) - d) / ( u - d)
    disc = np.exp(-rate * h) 
    spot_t = np.zeros((num_nodes, num_nodes))
    prc_t = np.zeros((num_nodes, num_nodes))
    del_t = np.zeros((num_nodes, num_nodes))
    bond_t = np.zeros((num_nodes, num_nodes))
    
    for j in range(num_nodes):
        spot_t[j,-1] = spot * (u ** (steps - j)) * (d ** (j))
        prc_t[j, -1] = option.payoff(spot_t[j, -1])

    for i in range((steps - 1), -1, -1):
        for j in range(i+1):
            spot_t[j,i] = spot_t[j,i+1] / u
            del_t[j,i] = (prc_t[j,i+1] - prc_t[j+1, i+1]) / (spot_t[j,i] * (u - d))
            bond_t[j,i] = disc * (u * prc_t[j+1,i+1] - d * prc_t[j,i+1])/ (u - d)
            prc_t[j,i] = del_t[j,i] * spot_t[j,i] + bond_t[j,i]
            
    return (prc_t[0,0], del_t[0,0])

In [7]:
spot = 104.0
strike = 95.0
rate = 0.08
expiry = 1.0
vol = 0.3
div = 0.0
u = 1.3
d = 0.8
steps = 3

In [9]:
spots = [80.0, 90.0, 110.0, 120.0, 130.0]

for spot in spots:
    the_call = VanillaOption(strike, expiry, call_payoff)
    prc, delta = american_binomial_prob4(the_call, spot, rate, vol, div, steps)
    print(f"({spot:0.2f}, {prc:0.2f}, {delta:0.2f})")

(80.00, 11.08, 0.48)
(90.00, 17.19, 0.61)
(110.00, 29.42, 0.79)
(120.00, 37.35, 0.84)
(130.00, 46.58, 0.88)
